In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
dataset = datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))

print(list(dataset.train_data.size()))
print("mean = ", dataset.train_data.float().mean()/255.0)
print("standard deviation = ", dataset.train_data.float().std()/255.0)


print("mean = ", dataset.test_data.float().mean()/255.0)
print("standard deviation = ", dataset.test_data.float().std()/255.0)

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!
[60000, 28, 28]


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


mean =  tensor(0.1306)
standard deviation =  tensor(0.3081)
mean =  tensor(0.1306)


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


standard deviation =  tensor(0.3081)


In [3]:
print(torch.__version__)
print(torch.cuda.is_available())

1.5.1+cu101
True


In [4]:
# sub class of nn.Module - 
# Base class for all neural network modules., All pytorch models should also subclass this class.
class Net(nn.Module):
    # constructor
    def __init__(self):
        # calling super class constructor
        super(Net, self).__init__()
        
        # https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF # 28, 28, 3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # 28, 28, 5
        self.pool1 = nn.MaxPool2d(2, 2) # 28, 14, 6
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # 14, 14, 10
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) # 14, 14, 14
        self.pool2 = nn.MaxPool2d(2, 2) # 14, 7, 16
        self.conv5 = nn.Conv2d(256, 512, 3) # 7, 5, 24
        self.conv6 = nn.Conv2d(512, 1024, 3) # 5, 3, 32
        self.conv7 = nn.Conv2d(1024, 10, 3) # 3, 1, 40

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # conv1 - relu - conv2 - relu - max pooling
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # conv3 - relu - cnov4 - relu - max pooling
        x = F.relu(self.conv6(F.relu(self.conv5(x)))) # conv5 - relu - conv6 - relu
        x = (self.conv7(x)) # conv7 - No need of relu
        x = x.view(-1, 10) # arranges the tensor in 10 rows / unpacking
        return F.log_softmax(x)
        # return F.softmax(x) 

In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available() 
device = torch.device("cuda" if use_cuda else "cpu")
print("use_cuda = ", use_cuda, ", device = ", device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

use_cuda =  True , device =  cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
---

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [6]:
torch.manual_seed(1) # intial seed
batch_size = 128 # batch size of input training data

# If you load your samples in the Dataset on CPU and would like to push it during training to the GPU, you can speed up the host to device transfer by enabling pin_memory.
# This lets your DataLoader allocate the samples in page-locked memory, which speeds-up the transfer.
# You can find more information on the NVIDIA blog 1.5k.

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
#            batch_sampler=None, num_workers=0, collate_fn=None,
#            pin_memory=False, drop_last=False, timeout=0,
#            worker_init_fn=None)

# training data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# test data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [7]:
# tqdm - progress bar
from tqdm import tqdm

# train method
def train(model, device, train_loader, optimizer, epoch):
    # model set for training
    model.train()

    pbar = tqdm(train_loader)
    
    for batch_idx, (data, target) in enumerate(pbar):
        # transfer data to the device - in this GPU
        data, target = data.to(device), target.to(device)
        # Sets gradients of all model parameters to zero.
        optimizer.zero_grad()
        
        # forward + backward + optimize
        output = model(data)

        # The negative log likelihood loss. It is useful to train a classification problem with C classes.
        # If provided, the optional argument weight should be a 1D Tensor assigning weight to each of the classes. 
        # This is particularly useful when you have an unbalanced training set.
        loss = F.nll_loss(output, target)

        # backward propagation -  computes dloss/dx for every parameter x which has requires_grad=True
        # These are accumulated into x.grad for every parameter x
        # x.grad += dloss/dx
        loss.backward()

        # updates the value of x using the gradient x.grad. 
        # x += -lr * x.grad
        optimizer.step()

        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


# test method
def test(model, device, test_loader):
    # start model for evaluation
    model.eval()
    test_loss = 0
    correct = 0
    # Context-manager that disabled gradient calculation.
    # Disabling gradient calculation is useful for inference, when you are sure that you will not call Tensor.backward(). 
    # It will reduce memory consumption for computations that would otherwise have requires_grad=True.
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            # calculate the output
            output = model(data)
            # calculate loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
# SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# run for 2 epochs
for epoch in range(1, 2):
    print(epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.03553972393274307 batch_id=468: 100%|██████████| 469/469 [00:37<00:00, 12.41it/s]



Test set: Average loss: 0.0655, Accuracy: 9796/10000 (98%)

